# The Applied Data Science Capstone Project Notebook

In [ ]:
## Exploration for 

## Introduction/Business Problem

A chef/restaurant owner would like to open a new restaurant in downtown Denver.   She wants to locate the restaurant near breweries since her food will complement nicely with local brews.  Her wish for a location is to be near breweries that are well rated and have many people going to them since that will help get people to her location.  The task here is to help find such a location.

## Data

We will leverage Foursquare results to explore breweries within a certain radius of location in downtown Denver.  After making the identification of these breweries, we will pull more information from Fourquare, including ratings, the number of 'likes', the number of pictures taken, and the number of 'likes'.  

In [1]:
import pandas as pd
import numpy as np